What I want initially is to use the whole 3000 stocks.
However due to the capacity of my laptop, now, I can only use one of them to form a baby-version model.

In [1]:
import numpy as np
import pandas as pd
import os
import copy

Due to the tradeoff of computing capacity and performance, I focus on two industries: Finance and Real Estate. 

In [2]:
finance=pd.read_excel('e:/data/finance.xlsx')
real_estate=pd.read_excel('e:/data/real_estate.xlsx')

In [3]:
fin=finance['finance']
rea=real_estate['real_estate']

I use the most resent 10 years' data to build my model.
And split the train and test set by proportion of 7:3, which means data from 2006-09-01 to 2013-09-02 is the training set, while data from 2013-09-02 to 2016-09-01 is the test set.

In [4]:
column_name=[]
for i in range(7):
    for j in ['clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff',
              'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value']:
        column_name.append(str(j)+'_lag'+str(i+1))

In [5]:
database={}
trainset={}
testset={}
for i in fin:
    
    #import my data
    
    name=str('0')+str(i)
    if os.path.isfile('e:/data/'+str(name)+'.csv'):
        data=pd.read_csv('e:/data/'+str(name)+'.csv',encoding='gb2312')
        data.columns=['date','code','name','clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff',
              'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value','hh']
        data=data.drop(['hh'],axis=1)

        #select the data

        if sum(data['date']=='2006-09-01')==1 and sum(data['date']=='2013-09-02')==1:

            #add some features

            for lag in range(7):     ## [1,2,3,4,5,6,7,8,9,10] will be better
                for col_name in ['clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff', 'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value']:
                    data[str(col_name)+'_lag'+str(lag+1)]=data[str(col_name)].shift(-lag-1)


                    #split the training set and the test set

            cutoff1=(data[data['date']=='2006-09-01'].index)
            cutoff2=(data[data['date']=='2013-09-02'].index)
            data=data[data.index<cutoff1[0]]
            train=data[data.index>cutoff2[0]]
            test =data[data.index<cutoff2[0]]
            database.update({str(name):data})
            trainset.update({'train_'+str(name):train})
            testset.update({'test_'+str(name):test})

In [6]:
for i in rea:
    
    #import my data
    
    name=str('0')+str(i)
    if os.path.isfile('e:/data/'+str(name)+'.csv'):
        data=pd.read_csv('e:/data/'+str(name)+'.csv',encoding='gb2312')
        data.columns=['date','code','name','clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff',
              'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value','hh']
        data=data.drop(['hh'],axis=1)

        #select the data

        if sum(data['date']=='2006-09-01')==1 and sum(data['date']=='2013-09-02')==1:

            #add some features

            for lag in range(7):     ## [1,2,3,4,5,6,7,8,9,10] will be better
                for col_name in ['clsPrc','highPrc','lowPrc','openPrc','clsPrc_lastday','diff', 'return', 'turnover_rate','volumn','turnover_value','total_market_value','current_market_value']:
                    data[str(col_name)+'_lag'+str(lag+1)]=data[str(col_name)].shift(-lag-1)


                    #split the training set and the test set

            cutoff1=(data[data['date']=='2006-09-01'].index)
            cutoff2=(data[data['date']=='2013-09-02'].index)
            data=data[data.index<cutoff1[0]]
            train=data[data.index>cutoff2[0]]
            test =data[data.index<cutoff2[0]]
            database.update({str(name):data})
            trainset.update({'train_'+str(name):train})
            testset.update({'test_'+str(name):test})

In [7]:
trainsetX={}
trainsety={}
testsetX={}
testsety={}

for code in database.keys():
    name=code
    train=trainset['train_'+str(name)]
    test=testset['test_'+str(name)]
    for i in column_name:
    
        train[i]=pd.to_numeric(train[i].values,errors='coerce')
        where_are_NaNs = np.isnan(train[i])
        train[str(i)][where_are_NaNs] = 0

        test[i]=pd.to_numeric(test[i].values,errors='coerce')
        where_are_NaNs = np.isnan(test[i])
        test[i][where_are_NaNs] = 0

    X_train=train[column_name]
    y_train=pd.to_numeric(train['diff'].values,errors='coerce')

    X_test=test[column_name]
    y_test=pd.to_numeric(test['diff'].values,errors='coerce')

    where_are_NaNs1 = np.isnan(y_train)
    y_train[where_are_NaNs1] = 0
    where_are_NaNs2 = np.isnan(y_test)
    y_test[where_are_NaNs2] = 0
    y_train=(y_train>0)*1
    y_test=(y_test>0)*1
    
    trainsetX.update({'trainX_'+str(name):X_train})
    trainsety.update({'trainy_'+str(name):y_train})
    testsetX.update({'testX_'+str(name):X_test})
    testsety.update({'testy_'+str(name):y_test})
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\si

In [117]:
pd.DataFrame(trainsetX,index=range(len(trainsetX['trainX_0600000']['clsPrc_lag1'])))

ValueError: cannot copy sequence with size 84 to array axis with dimension 1699

In [8]:
# 1. import
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

for code in database.keys():
    name=code
    X_train=trainsetX['trainX_'+str(name)]
    y_train=trainsety['trainy_'+str(name)]
    X_test=testsetX['testX_'+str(name)]
    y_test=testsety['testy_'+str(name)]

    # 2. instantiate
    mlp = MLPClassifier()

    # 3. train
    mlp.fit(X_train, y_train)

    # 4. predict
    p = mlp.predict(X_test)

    # 5. evaluate
    print(code)
    print(metrics.accuracy_score(p, y_test))

0600291
0.56070941337
0600318
0.678035470668
0600816
0.485675306958
0600015
0.532060027285
0600155
0.335607094134
0600705
0.429740791269
0600643
0.421555252387
0600000
0.512960436562
0600061
0.440654843111
0600695
0.515688949523
0600837
0.489768076398
0600016
0.465211459754
0600369
0.534788540246
0600036
0.482946793997
0600030
0.514324693042
0600109
0.49043715847
0600621
0.512295081967
0600390
0.526603001364
0600599
0.43519781719
0601988
0.446111869031
0600162
0.491132332879
0600067
0.518417462483
0600807
0.506139154161
0600240
0.538881309686
0600159
0.474761255116
0600223
0.493860845839
0600533
0.441256830601
0600064
0.488403819918
0600736
0.534788540246
0600510
0.485675306958
0600053
0.653005464481
0600077
0.517053206003
0600716
0.507503410641
0600173
0.43929058663
0600340
0.49795361528
0600325
0.506139154161
0600503
0.551159618008
0600724
0.496589358799
0600158
0.484311050477
0600322
0.519781718963
0600649
0.380627557981
0600638
0.510231923602
0600604
0.470668485675
0600732
0.372424